 <font color='red'> DISCLAIMER </font> 
 
 
 I do not pretend to have fully commented nor have produced a perfect code.
 Use to your  <font color='red'> own risks </font> 

In [1]:
# ----- Librairies ----- #
import pandas as pd
import re
import sys
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tag import UnigramTagger
from nltk.corpus import brown
import webcolors
from IPython.display import display

import matplotlib.pyplot as plt
import math

from sklearn.neighbors import NearestNeighbors

sys.path.append("..")
from ADA_JEX2017.Project.Functions.str_functions import *
from ADA_JEX2017.Project.Functions.pre_process import *

In [2]:
path='../ADA_JEX2017/Project/Functions/'
#our datafile exported to csv
# ----- Loading the dataset'recipeInfo_WestWhiteHorvitz_WWW2013.csv' ----- #
data_file='./recipeInfo/recipeInfo_WestWhiteHorvitz_WWW2013_v2.csv'
raw_data = pd.read_csv(data_file ,sep=';')

In [6]:
# ----- Initializing and loading the list of techniques, units and ingredients created previously ----- #
with open(path+'units_list.txt', 'r') as f:
    units_list = [line.rstrip('\n') for line in f]
    
with open(path+'technique_list.txt', 'r') as f:
    techniques_list = [line.rstrip('\n') for line in f]

with open(path+'ingredient_list.txt', 'r') as f:
    ingredient_list = [line.rstrip('\n') for line in f]



In [19]:
# ----- Make a dataframe with our data while dropping the NaN values ----- #
ingr_dataframe=raw_data[['title','ingredients_list']].copy().dropna()
ingr_dataframe = ingr_dataframe.reset_index(drop=True)
display(ingr_dataframe.head())
print(len(ingr_dataframe))
# Ignore upper case in the ingredients list string
ingr_dataframe['ingredients_list']=ingr_dataframe['ingredients_list'].str.lower()

#ingr_data_reduced=ingr_dataframe.head(100) # create a reduced data as draft to test when creating new functions

,title,ingredients_list
0,Easy Light Chocolate Milkshake Recipe,"put one half cup of milk, 4 tablespoons of cho..."
1,Lamb Stew Recipe : : Recipes : Food Network,5 yellow onions|2 turnips|5 carrots|1 stalk fe...
2,Chocolate Bread Pudding Recipe : Paula Deen : ...,"1 (1-pound) loaf French or Italian bread, cube..."
3,Snowball Cookies II Recipe,1/2 cup powdered sugar|1/3 cup butter or marga...
4,Old Fashioned Butterscotch Pie Recipe #31698,1 1/2 cup brown sugar|1 cup water|3 eggs|4 tab...


46277


In [20]:
# ----- Function to process the text in the ingredient list ----- # cf pre_process.py
# We notice that for some ingredients in the ingredients list, the quantity is given twice with one quantity given in volume or mass within parenthesis
# Therefore, we apply the next function to return only the wanted quantity
fun_add_preprocess(ingr_dataframe,units_list)

## Create gram values

In [21]:
#Load the dataframe webscrapped
unit_scraped=pd.read_csv('bbc_scraped.csv','\t').dropna(how='all',axis=1)
unit_scraped.set_index('Unnamed: 0',inplace=True)
#unit_scraped.count(axis=0)
#load the list created
unit_created=pd.read_csv('BBC_unit.csv',sep=';')
unit_created.set_index('Unnamed: 0',inplace=True)

In [22]:
#Could be enhanced (manually or scapping)
#still a lot of ingredient have a default value of 200

ingred_unit={}
for index, row  in unit_scraped.iterrows():
    units=[]
    for i,quant in enumerate(list(row.dropna().index)):
        quant=quant.replace(',','').lower()
        if any(q in units_list+['slice']+['nlea'] for q in quant.split(' ') if len(q)>0):
            a=1
        else:
            gram_val=float([re.sub('[^0-9]','',  row.dropna().values[i])][0])
            units.append(gram_val)
    if len(units)==0:
        units=200
    ingred_unit[index]=np.mean(units)
    #print(list(row.dropna().index))
    
for index, row in unit_created.iterrows():
    if index in ingred_unit:
        print('Problem: it was a unit')
    ingred_unit[index]=row['1 Unit']
ingred_unit['clove']=2
ingred_unit['egg']=50    

In [37]:
# !!!!! ----- Test cell : to inspect a specific recipe ----- !!!!!! #
def test_receipe(k):
    receipe=ingr_dataframe.loc[k]['Recipe_preporcess']
    print(receipe)
    dic_ingr,dictec,wasted,wasted_numb=fun_extract_ingredients\
        (receipe,ingredient_list,techniques_list,units_list,ingred_unit\
         ,to_gram=True)
    return dic_ingr
test_receipe(28)

1/3 cup quinoa|1 cup water|1 pinch salt|1 tablespoon olive oil|1 teaspoon minced garlic|1 small sweet onion, chopped|1 cup crimini mushrooms, sliced|1 small sweet potato, peeled and diced|1/4 teaspoon cayenne pepper|salt and pepper to taste|1/4 cup chopped, toasted pecans


{'cayenne pepper': 1.25,
 'garlic': 5.0,
 'mushroom': 237.0,
 'olive oil': 15.0,
 'onion': 97.5,
 'pecan': 59.25,
 'pepper': 51.3333333333,
 'quinoa': 79.0,
 'salt': 0.36,
 'sweet potato': 200.0,
 'water': 237.0}

In [31]:
#----- Use whole data frame to extract each ingredient with its quantity and unit by using the lists  ------ #

ingr_data_reduced=ingr_dataframe.head(100)
all_dic=[]
not_used_ingr=[]
wastes=0
for index, row in ingr_dataframe.iterrows():
    recipe=row['Recipe_preporcess']

    # Function in str_functions.py to extract the ingredients for each recipe
    dic_ingre,dictec,wasted_ingr,wasted_number=fun_extract_ingredients\
            (recipe,ingredient_list,techniques_list,units_list,ingred_unit,to_gram=True)
    # Also convert each quantity in the same unit (grams) if to_gram is set to True
    all_dic.append(dic_ingre)

# We implemented the number of ingredients which didn't fit the criteria 
# Then we plotted the ingredient that we threw away in order to complete manually our ingredient list with important ingredients that our list may miss
    #not_used_ingr.append(wasted_ingr) 
    #wastes=wastes+wasted_number
    
# ----- Create the dataframe of all the ingredient and their quantities ----- #
ingredients_frame=pd.DataFrame(data=all_dic)
display(ingredients_frame.head(5))

# ----- Print the number of ingredients ----- #
print('There are : ',len(list(ingredients_frame)), 'ingredients')
ingred_used={}
for i in list(ingredients_frame):
    ingred_used[i]=sum(ingredients_frame[i].value_counts())

,acidulated water,acorn squash,agave,ale,aleppo pepper,alfalfa sprout,allspice,almond,almond extract,almond milk,...,worcestershire sauce,wrap,yam,yeast,yellow lentil,yoghurt,yogurt,yuzu,zest,zucchini
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.625,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are :  861 ingredients
